# Librerias

In [1]:
import requests
import pandas as pd
from datetime import datetime
import time
import os
import base64
import json
import numpy as np

In [2]:
save_directory = './pokemon-sprites/'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Extraer

In [3]:
url = 'https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0'
response = requests.get(url)
data = response.json()

In [5]:
list_pkmn_primary_data = []
list_abilities = []
list_game_appears = []
list_stats = []
list_types = []
list_pokedex_description = []
list_evolutions = []

for pokemon in data['results']:
    response_info_pokemon = requests.get(pokemon['url'])
    data_info_pokemon = response_info_pokemon.json()
    
    # Datos principales
    id_pokemon = data_info_pokemon['id']
    pokemon_name = pokemon['name']
    pokemon_order = data_info_pokemon['order']
    pokemon_weight = data_info_pokemon['weight']
    pokemon_height = data_info_pokemon['height']
    
    # Datos de habilidades
    pokemon_abilities = data_info_pokemon['abilities']
    for ability in pokemon_abilities:
        ability_name = ability['ability']['name']
        ability_is_hidden = ability['is_hidden']
        # URL de la habildiad para obtener la descripción
        ability_url = ability['ability']['url']
        response_ability_pokemon = requests.get(ability_url)
        data_ability_pokemon = response_ability_pokemon.json()
        id_ability = data_ability_pokemon['id']
        # Obtener la descripcion en ingles de la la habildiad
        for ability in data_ability_pokemon['flavor_text_entries']:
            if ability['language']['name'] == 'en':
                ability_description = ability['flavor_text']
        
        list_abilities.append([id_pokemon, id_ability, ability_name, ability_is_hidden, ability_description])
    
    # Datos Especie
    species_url = data_info_pokemon['species']['url']
    response_species = requests.get(species_url)
    data_species = response_species.json()
    flavor_text_entries = data_species['flavor_text_entries']
    for entry in flavor_text_entries:
        if entry['language']['name'] == 'en':
            flavor_text = entry['flavor_text']
            flavor_text = flavor_text.replace("\n", ' ')
            game_name = entry['version']['name']
        list_pokedex_description.append([id_pokemon, flavor_text, game_name])

    # Datos Evolucion
    evolution_chain_url = data_species['evolution_chain']['url']
    response_evolution_chain = requests.get(evolution_chain_url)
    data_evolution_chain = response_evolution_chain.json()
    evolve_to_pkmn = data_evolution_chain['chain']['species']['name']
    evol_req_gender = ''
    evol_req_held_item = ''
    evol_req_item = ''
    evol_req_known_move = ''
    evol_req_known_move_type = ''
    evol_req_location = ''
    evol_req_min_affection = ''
    evol_req_min_beauty = ''
    evol_req_min_happiness = ''
    evol_req_min_level = ''
    evol_req_needs_overworld_rain = ''
    evol_req_party_species = ''
    evol_req_party_type = ''
    evol_req_relative_physical_stats = ''
    evol_req_time_of_day = ''
    evol_req_trade_species = ''
    evol_req_turn_upside_down = ''
    evol_req_baby = True
    list_evolutions.append([id_pokemon,
                evolve_to_pkmn, evol_req_gender, evol_req_held_item
                , evol_req_item, evol_req_known_move, evol_req_known_move_type
                , evol_req_location, evol_req_min_affection, evol_req_min_beauty
                , evol_req_min_happiness, evol_req_min_level, evol_req_needs_overworld_rain
                , evol_req_party_species, evol_req_party_type, evol_req_relative_physical_stats
                , evol_req_time_of_day, evol_req_trade_species, evol_req_turn_upside_down,evol_req_baby
            ])
    if len(data_evolution_chain['chain']['evolves_to']) > 0:
        for evolve_to_1 in data_evolution_chain['chain']['evolves_to']: 
            evolve_to_pkmn = evolve_to_1['species']['name']
            if len(evolve_to_1['evolution_details']) > 0:
                evol_req_gender = evolve_to_1['evolution_details'][0]['gender']
                evol_req_held_item = evolve_to_1['evolution_details'][0]['held_item']
                evol_req_item = evolve_to_1['evolution_details'][0]['item']
                evol_req_known_move = evolve_to_1['evolution_details'][0]['known_move']
                evol_req_known_move_type = evolve_to_1['evolution_details'][0]['known_move_type']
                evol_req_location = evolve_to_1['evolution_details'][0]['location']
                evol_req_min_affection = evolve_to_1['evolution_details'][0]['min_affection']
                evol_req_min_beauty = evolve_to_1['evolution_details'][0]['min_beauty']
                evol_req_min_happiness = evolve_to_1['evolution_details'][0]['min_happiness']
                evol_req_min_level = evolve_to_1['evolution_details'][0]['min_level']
                evol_req_needs_overworld_rain = evolve_to_1['evolution_details'][0]['needs_overworld_rain']
                evol_req_party_species = evolve_to_1['evolution_details'][0]['party_species']
                evol_req_party_type = evolve_to_1['evolution_details'][0]['party_type']
                evol_req_relative_physical_stats = evolve_to_1['evolution_details'][0]['relative_physical_stats']
                evol_req_time_of_day = evolve_to_1['evolution_details'][0]['time_of_day']
                evol_req_trade_species = evolve_to_1['evolution_details'][0]['trade_species']
                evol_req_turn_upside_down = evolve_to_1['evolution_details'][0]['turn_upside_down']
                evol_req_baby = False
            else:
                evol_req_gender = ''
                evol_req_held_item = ''
                evol_req_item = ''
                evol_req_known_move = ''
                evol_req_known_move_type = ''
                evol_req_location = ''
                evol_req_min_affection = ''
                evol_req_min_beauty = ''
                evol_req_min_happiness = ''
                evol_req_min_level = ''
                evol_req_needs_overworld_rain = ''
                evol_req_party_species = ''
                evol_req_party_type = ''
                evol_req_relative_physical_stats = ''
                evol_req_time_of_day = ''
                evol_req_trade_species = ''
                evol_req_turn_upside_down = ''
                evol_req_baby = False
            list_evolutions.append([id_pokemon,
                evolve_to_pkmn, evol_req_gender, evol_req_held_item
                , evol_req_item, evol_req_known_move, evol_req_known_move_type
                , evol_req_location, evol_req_min_affection, evol_req_min_beauty
                , evol_req_min_happiness, evol_req_min_level, evol_req_needs_overworld_rain
                , evol_req_party_species, evol_req_party_type, evol_req_relative_physical_stats
                , evol_req_time_of_day, evol_req_trade_species, evol_req_turn_upside_down, evol_req_baby
            ])
            if len(evolve_to_1['evolves_to']) > 0:
                for evolve_to_2 in evolve_to_1['evolves_to']:
                    evolve_to_pkmn = evolve_to_2['species']['name']
                    if len(evolve_to_2['evolution_details']) > 0:
                        evol_req_gender = evolve_to_2['evolution_details'][0]['gender']
                        evol_req_held_item = evolve_to_2['evolution_details'][0]['held_item']
                        evol_req_item = evolve_to_2['evolution_details'][0]['item']
                        evol_req_known_move = evolve_to_2['evolution_details'][0]['known_move']
                        evol_req_known_move_type = evolve_to_2['evolution_details'][0]['known_move_type']
                        evol_req_location = evolve_to_2['evolution_details'][0]['location']
                        evol_req_min_affection = evolve_to_2['evolution_details'][0]['min_affection']
                        evol_req_min_beauty = evolve_to_2['evolution_details'][0]['min_beauty']
                        evol_req_min_happiness = evolve_to_2['evolution_details'][0]['min_happiness']
                        evol_req_min_level = evolve_to_2['evolution_details'][0]['min_level']
                        evol_req_needs_overworld_rain = evolve_to_2['evolution_details'][0]['needs_overworld_rain']
                        evol_req_party_species = evolve_to_2['evolution_details'][0]['party_species']
                        evol_req_party_type = evolve_to_2['evolution_details'][0]['party_type']
                        evol_req_relative_physical_stats = evolve_to_2['evolution_details'][0]['relative_physical_stats']
                        evol_req_time_of_day = evolve_to_2['evolution_details'][0]['time_of_day']
                        evol_req_trade_species = evolve_to_2['evolution_details'][0]['trade_species']
                        evol_req_turn_upside_down = evolve_to_2['evolution_details'][0]['turn_upside_down']
                        evol_req_baby = False
                    else:
                        evol_req_gender = ''
                        evol_req_held_item = ''
                        evol_req_item = ''
                        evol_req_known_move = ''
                        evol_req_known_move_type = ''
                        evol_req_location = ''
                        evol_req_min_affection = ''
                        evol_req_min_beauty = ''
                        evol_req_min_happiness = ''
                        evol_req_min_level = ''
                        evol_req_needs_overworld_rain = ''
                        evol_req_party_species = ''
                        evol_req_party_type = ''
                        evol_req_relative_physical_stats = ''
                        evol_req_time_of_day = ''
                        evol_req_trade_species = ''
                        evol_req_turn_upside_down = ''
                        evol_req_baby = False
                    list_evolutions.append([id_pokemon,
                        evolve_to_pkmn, evol_req_gender, evol_req_held_item
                        , evol_req_item, evol_req_known_move, evol_req_known_move_type
                        , evol_req_location, evol_req_min_affection, evol_req_min_beauty
                        , evol_req_min_happiness, evol_req_min_level, evol_req_needs_overworld_rain
                        , evol_req_party_species, evol_req_party_type, evol_req_relative_physical_stats
                        , evol_req_time_of_day, evol_req_trade_species, evol_req_turn_upside_down, evol_req_baby
                    ])

    # Datos de apariciones en versiones
    pokemon_game_indices = data_info_pokemon['game_indices']
    for game in pokemon_game_indices:
        game_name = game['version']['name']
        list_game_appears.append([id_pokemon, game_name])

    # Datos de estadistica
    pokemon_stats = data_info_pokemon['stats']
    for stat in pokemon_stats:
        stat_name = stat['stat']['name']
        stat_base = stat['base_stat']
        stat_effort = stat['effort']
        list_stats.append([id_pokemon, stat_name, stat_base, stat_effort])
            
    # Datos de tipo
    pokemon_types = data_info_pokemon['types']
    for type in pokemon_types:
        type_name = type['type']['name']
        type_number = type['slot']
        list_types.append([id_pokemon, type_name, type_number])

    # Descargar el sprite por defecto del pokemon
    front_default_url = data_info_pokemon['sprites']['front_default']
    if front_default_url:
        image_name = f'{id_pokemon}-{pokemon_name}.png'
        image_path = os.path.join(save_directory, image_name)
        img_response = requests.get(front_default_url)

        if img_response.status_code == 200:
            with open(image_path, 'wb') as img_file:
                img_file.write(img_response.content)
                list_pkmn_primary_data.append([id_pokemon, pokemon_order, pokemon_name, pokemon_weight, pokemon_height, image_path])
                print("Registro exitoso: " + pokemon_name)
        else:
            print('Registro exitoso ' + pokemon_name + ' pero falló la descarga del sprite')
    else:
        print('Registro exitoso ' + pokemon_name + ' pero no existe sprite')
        image_path = ''
        list_pkmn_primary_data.append([id_pokemon, pokemon_order, pokemon_name, pokemon_weight, pokemon_height, image_path])
    
    time.sleep(0.1)
    

Registro exitoso: bulbasaur
Registro exitoso: ivysaur
Registro exitoso: venusaur
Registro exitoso: charmander
Registro exitoso: charmeleon
Registro exitoso: charizard
Registro exitoso: squirtle
Registro exitoso: wartortle
Registro exitoso: blastoise
Registro exitoso: caterpie
Registro exitoso: metapod
Registro exitoso: butterfree
Registro exitoso: weedle
Registro exitoso: kakuna
Registro exitoso: beedrill
Registro exitoso: pidgey
Registro exitoso: pidgeotto
Registro exitoso: pidgeot
Registro exitoso: rattata
Registro exitoso: raticate
Registro exitoso: spearow
Registro exitoso: fearow
Registro exitoso: ekans
Registro exitoso: arbok
Registro exitoso: pikachu
Registro exitoso: raichu
Registro exitoso: sandshrew
Registro exitoso: sandslash
Registro exitoso: nidoran-f
Registro exitoso: nidorina
Registro exitoso: nidoqueen
Registro exitoso: nidoran-m
Registro exitoso: nidorino
Registro exitoso: nidoking
Registro exitoso: clefairy
Registro exitoso: clefable
Registro exitoso: vulpix
Registro 

In [6]:
df_pkmn_primary_data_extraer = pd.DataFrame(
    list_pkmn_primary_data,
    columns = ['pkmn_id', 'pokemon_order', 'pokemon_name', 'pokemon_weight', 'pokemon_height', 'pokemon_image_path']
)

df_pkmn_abilities_extraer = pd.DataFrame(
    list_abilities,
    columns = ['pkmn_id', 'ability_id', 'ability_name', 'ability_is_hidden', 'ability_description']
)

df_pkmn_game_appears_extraer = pd.DataFrame(
    list_game_appears,
    columns = ['pkmn_id', 'game_version']
)

df_pkmn_stats_extraer = pd.DataFrame(
    list_stats,
    columns = ['pkmn_id', 'name', 'stat', 'effort']
)

df_pkmn_types_extraer = pd.DataFrame(
    list_types,
    columns = ['pkmn_id', 'name', 'type_number']
)

df_pkmn_pokedex_description_extraer = pd.DataFrame(
    list_pokedex_description,
    columns = ['pkmn_id', 'flavor_text', 'game_name']
)

df_pkmn_evolution_extraer = pd.DataFrame(
    list_evolutions,
    columns = [
        'pkmn_id', 'evolve_to_pkmn', 'evol_req_gender', 'evol_req_held_item'
        , 'evol_req_item', 'evol_req_known_move', 'evol_req_known_move_type'
        , 'evol_req_location', 'evol_req_min_affection', 'evol_req_min_beauty'
        , 'evol_req_min_happiness', 'evol_req_min_level', 'evol_req_needs_overworld_rain'
        , 'evol_req_party_species', 'evol_req_party_type', 'evol_req_relative_physical_stats'
        , 'evol_req_time_of_day', 'evol_req_trade_species', 'evol_req_turn_upside_down', 'evol_req_baby'
        ]
)

In [ ]:
df_pkmn_evolution_extraer.head(100)

In [7]:
df_pkmn_primary_data_extraer['extraer_datetime'] = datetime.now()
df_pkmn_abilities_extraer['extraer_datetime'] = datetime.now()
df_pkmn_game_appears_extraer['extraer_datetime'] = datetime.now()
df_pkmn_stats_extraer['extraer_datetime'] = datetime.now()
df_pkmn_types_extraer['extraer_datetime'] = datetime.now()
df_pkmn_pokedex_description_extraer['extraer_datetime'] = datetime.now()
df_pkmn_evolution_extraer['extraer_datetime'] = datetime.now()

In [8]:
df_pkmn_primary_data_extraer.to_csv('fuente/extraer/pkmn_primary_data.csv', index = False, sep = '|')
df_pkmn_abilities_extraer.to_csv('fuente/extraer/pkmn_abilities.csv', index = False, sep = '|')
df_pkmn_game_appears_extraer.to_csv('fuente/extraer/pkmn_game_appears.csv', index = False, sep = '|')
df_pkmn_stats_extraer.to_csv('fuente/extraer/pkmn_stats.csv', index = False, sep = '|')
df_pkmn_types_extraer.to_csv('fuente/extraer/pkmn_types.csv', index = False, sep = '|')
df_pkmn_pokedex_description_extraer.to_csv('fuente/extraer/pkmn_pokedex_description.csv', index = False, sep= '|')
df_pkmn_evolution_extraer.to_csv('fuente/extraer/pkmn_evolution.csv', index= False, sep='|')

# Transformar

## Leer Data

In [9]:
df_pkmn_primary_data_transformar = pd.read_csv('fuente/extraer/pkmn_primary_data.csv', delimiter = '|')
df_pkmn_abilities_transformar = pd.read_csv('fuente/extraer/pkmn_abilities.csv', delimiter = '|')
df_pkmn_types_transformar = pd.read_csv('fuente/extraer/pkmn_types.csv', delimiter= '|')
df_pkmn_game_appears_transformar = pd.read_csv('fuente/extraer/pkmn_game_appears.csv', delimiter='|')
df_pkmn_stats_transformar = pd.read_csv('fuente/extraer/pkmn_stats.csv', delimiter='|')
df_pkmn_pokedex_description_transformar = pd.read_csv('fuente/extraer/pkmn_pokedex_description.csv', delimiter = '|')
df_pkmn_evolution_transformar = pd.read_csv('fuente/extraer/pkmn_evolution.csv', delimiter='|')

In [10]:
# Pkmn Data
df_pkmn_primary_data_transformar = df_pkmn_primary_data_transformar[['pkmn_id', 'pokemon_order', 'pokemon_name', 'pokemon_weight', 'pokemon_height', 'pokemon_image_path']]
# Pkmn abilities
df_pkmn_abilities_transformar = df_pkmn_abilities_transformar[['pkmn_id', 'ability_id', 'ability_name', 'ability_is_hidden', 'ability_description']]
# Pkmn Types
df_pkmn_types_transformar = df_pkmn_types_transformar[['pkmn_id', 'name', 'type_number']]
# Pkmn Gamer Appears
df_pkmn_game_appears_transformar = df_pkmn_game_appears_transformar[['pkmn_id', 'game_version']]
# Pkmn Stats
df_pkmn_stats_transformar = df_pkmn_stats_transformar[['pkmn_id', 'name', 'stat', 'effort']]
# Pkmn Pokedex Description
df_pkmn_pokedex_description_transformar['id_pokedex_description'] = range(1, len(df_pkmn_pokedex_description_transformar) + 1)
df_pkmn_pokedex_description_transformar = df_pkmn_pokedex_description_transformar[['id_pokedex_description', 'pkmn_id', 'game_name', 'flavor_text']]
# Pkmn evolution
df_pkmn_evolution_transformar = df_pkmn_evolution_transformar[['pkmn_id','evolve_to_pkmn', 'evol_req_gender', 'evol_req_held_item', 'evol_req_item', 'evol_req_known_move', 'evol_req_known_move_type', 'evol_req_location', 'evol_req_min_affection', 'evol_req_min_beauty', 'evol_req_min_happiness', 'evol_req_min_level', 'evol_req_needs_overworld_rain', 'evol_req_party_species', 'evol_req_party_type', 'evol_req_relative_physical_stats', 'evol_req_time_of_day', 'evol_req_trade_species', 'evol_req_turn_upside_down', 'evol_req_baby']]


## Limpiar tabla evolution

### Limpiar duplicados y agregar indice

In [294]:
# df_pkmn_evolution_transformar = df_pkmn_evolution_transformar.drop_duplicates()
# df_pkmn_evolution_transformar['id_pkmn_evolution'] = range(1, len(df_pkmn_evolution_transformar)+ 1)
# df_pkmn_evolution_transformar = df_pkmn_evolution_transformar[['id_pkmn_evolution','','evolve_to_pkmn', 'evol_req_gender', 'evol_req_held_item', 'evol_req_item', 'evol_req_known_move', 'evol_req_known_move_type', 'evol_req_location', 'evol_req_min_affection', 'evol_req_min_beauty', 'evol_req_min_happiness', 'evol_req_min_level', 'evol_req_needs_overworld_rain', 'evol_req_party_species', 'evol_req_party_type', 'evol_req_relative_physical_stats', 'evol_req_time_of_day', 'evol_req_trade_species', 'evol_req_turn_upside_down']]

In [11]:
df_pkmn_evolution_transformar.head()

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
0,1,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False
2,1,venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False,False
3,2,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False


### Analizar cada campo de la tabla evolution

#### Campo evol_req_gender
- Los valores 1 representan al requerimiento de que el pokemon debe ser de género hembra
- Los valores 2 representan al requerimiento de que el pokemon debe ser de género macho

In [12]:
df_pkmn_evolution_transformar.groupby('evol_req_gender').size().reset_index(name='count')

,evol_req_gender,count
0,1.0,14
1,2.0,11


El evol_req_gender = 1 son pokemon de genero hembra

In [13]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_gender'] == 1]

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
913,361,froslass,1.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
916,362,froslass,1.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
1033,412,wormadam,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1036,413,wormadam,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1039,414,wormadam,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1042,415,vespiquen,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1044,416,vespiquen,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1195,478,froslass,1.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
1808,757,salazzle,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1810,758,salazzle,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0,False,NaN,NaN,NaN,NaN,NaN,False,False


El evol_req_gender = 2 son pokemon de genero macho

In [14]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_gender'] == 2]

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
743,280,gallade,2.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
747,281,gallade,2.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
751,282,gallade,2.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
1034,412,mothim,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1037,413,mothim,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1040,414,mothim,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,NaN,NaN,NaN,False,False
1187,475,gallade,2.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
2350,10004,mothim,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,NaN,NaN,NaN,False,False
2353,10005,mothim,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,NaN,NaN,NaN,False,False
2435,10051,gallade,2.0,NaN,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False


Tenerlo en consideración para crear una tabla con ese indice y valor

#### Campo evol_req_held_item

Dentro del evol_req_held contiene un json con la información de cierto item que require el pokemon tenerlo para que pueda evolucionar

In [15]:
df_pkmn_evolution_transformar.groupby('evol_req_held_item').size().reset_index(name='count')

,evol_req_held_item,count
0,"{'name': 'deep-sea-scale', 'url': 'https://pok...",3
1,"{'name': 'deep-sea-tooth', 'url': 'https://pok...",3
2,"{'name': 'dragon-scale', 'url': 'https://pokea...",3
3,"{'name': 'dubious-disc', 'url': 'https://pokea...",3
4,"{'name': 'electirizer', 'url': 'https://pokeap...",3
5,"{'name': 'kings-rock', 'url': 'https://pokeapi...",11
6,"{'name': 'magmarizer', 'url': 'https://pokeapi...",3
7,"{'name': 'metal-coat', 'url': 'https://pokeapi...",7
8,"{'name': 'oval-stone', 'url': 'https://pokeapi...",3
9,"{'name': 'protector', 'url': 'https://pokeapi....",3


In [16]:
def extract_name_from_json(val):
    if pd.notna(val):  # Verifica si el valor no es NaN
        try:
            # Convierte la cadena JSON a un diccionario si es necesario
            json_obj = json.loads(val.replace("'", "\""))
            # Extrae el valor de la llave 'name'
            return json_obj.get('name', np.nan)
        except (json.JSONDecodeError, TypeError):
            # Si falla la conversión, devuelve NaN
            return np.nan
    return np.nan  # Devuelve NaN si el valor es NaN

In [17]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_held_item'] = df_pkmn_evolution_transformar['evol_req_held_item'].apply(extract_name_from_json)

In [18]:
df_pkmn_evolution_transformar.groupby('evol_req_held_item').size().reset_index(name='count')

,evol_req_held_item,count
0,deep-sea-scale,3
1,deep-sea-tooth,3
2,dragon-scale,3
3,dubious-disc,3
4,electirizer,3
5,kings-rock,11
6,magmarizer,3
7,metal-coat,7
8,oval-stone,3
9,protector,3


#### Campo evol_req_item

In [19]:
df_pkmn_evolution_transformar.groupby('evol_req_item').size().reset_index(name='count')

,evol_req_item,count
0,"{'name': 'auspicious-armor', 'url': 'https://p...",3
1,"{'name': 'black-augurite', 'url': 'https://pok...",4
2,"{'name': 'cracked-pot', 'url': 'https://pokeap...",2
3,"{'name': 'dawn-stone', 'url': 'https://pokeapi...",10
4,"{'name': 'dusk-stone', 'url': 'https://pokeapi...",11
5,"{'name': 'fire-stone', 'url': 'https://pokeapi...",23
6,"{'name': 'ice-stone', 'url': 'https://pokeapi....",2
7,"{'name': 'leaf-stone', 'url': 'https://pokeapi...",15
8,"{'name': 'malicious-armor', 'url': 'https://po...",3
9,"{'name': 'moon-stone', 'url': 'https://pokeapi...",16


In [20]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_item'] = df_pkmn_evolution_transformar['evol_req_item'].apply(extract_name_from_json)

In [21]:
df_pkmn_evolution_transformar.groupby('evol_req_item').size().reset_index(name='count')

,evol_req_item,count
0,auspicious-armor,3
1,black-augurite,4
2,cracked-pot,2
3,dawn-stone,10
4,dusk-stone,11
5,fire-stone,23
6,ice-stone,2
7,leaf-stone,15
8,malicious-armor,3
9,moon-stone,16


#### Campo evol_req_known_move

In [22]:
df_pkmn_evolution_transformar.groupby('evol_req_known_move').size().reset_index(name='count')

,evol_req_known_move,count
0,"{'name': 'ancient-power', 'url': 'https://poke...",7
1,"{'name': 'barb-barrage', 'url': 'https://pokea...",3
2,"{'name': 'double-hit', 'url': 'https://pokeapi...",2
3,"{'name': 'dragon-pulse', 'url': 'https://pokea...",2
4,"{'name': 'hyper-drill', 'url': 'https://pokeap...",3
5,"{'name': 'mimic', 'url': 'https://pokeapi.co/a...",6
6,"{'name': 'psyshield-bash', 'url': 'https://pok...",2
7,"{'name': 'rollout', 'url': 'https://pokeapi.co...",2
8,"{'name': 'stomp', 'url': 'https://pokeapi.co/a...",3
9,"{'name': 'taunt', 'url': 'https://pokeapi.co/a...",2


In [23]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_known_move'] = df_pkmn_evolution_transformar['evol_req_known_move'].apply(extract_name_from_json)

In [24]:
df_pkmn_evolution_transformar.groupby('evol_req_known_move').size().reset_index(name='count')

,evol_req_known_move,count
0,ancient-power,7
1,barb-barrage,3
2,double-hit,2
3,dragon-pulse,2
4,hyper-drill,3
5,mimic,6
6,psyshield-bash,2
7,rollout,2
8,stomp,3
9,taunt,2


#### Campo evol_req_known_move_type

In [25]:
df_pkmn_evolution_transformar.groupby('evol_req_known_move_type').size().reset_index(name='count')

,evol_req_known_move_type,count
0,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",11


In [26]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_known_move_type'].notna()]

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
347,133,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
356,134,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
365,135,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
374,136,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
528,196,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
537,197,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
1166,470,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
1175,471,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
1688,700,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
2656,10159,sylveon,NaN,NaN,NaN,NaN,"{'name': 'fairy', 'url': 'https://pokeapi.co/a...",NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False


In [27]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_known_move_type'] = df_pkmn_evolution_transformar['evol_req_known_move_type'].apply(extract_name_from_json)

In [28]:
df_pkmn_evolution_transformar.groupby('evol_req_known_move_type').size().reset_index(name='count')

,evol_req_known_move_type,count
0,fairy,11


#### Campo evol_req_location

In [29]:
df_pkmn_evolution_transformar.groupby('evol_req_location').size().reset_index(name='count')

,evol_req_location,count
0,"{'name': 'eterna-forest', 'url': 'https://poke...",11
1,"{'name': 'mount-lanakila', 'url': 'https://pok...",2
2,"{'name': 'mt-coronet', 'url': 'https://pokeapi...",5
3,"{'name': 'sinnoh-route-217', 'url': 'https://p...",11


In [30]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_location'] = df_pkmn_evolution_transformar['evol_req_location'].apply(extract_name_from_json)

In [31]:
df_pkmn_evolution_transformar.groupby('evol_req_location').size().reset_index(name='count')

,evol_req_location,count
0,eterna-forest,11
1,mount-lanakila,2
2,mt-coronet,5
3,sinnoh-route-217,11


#### Campo evol_req_min_affection

In [32]:
df_pkmn_evolution_transformar.groupby('evol_req_min_affection').size().reset_index(name='count')

,evol_req_min_affection,count
0,2.0,11


#### Campo evol_req_min_beauty

In [33]:
df_pkmn_evolution_transformar.groupby('evol_req_min_beauty').size().reset_index(name='count')

,evol_req_min_beauty,count
0,171.0,2


#### Campo evol_req_min_happiness

In [34]:
df_pkmn_evolution_transformar.groupby('evol_req_min_happiness').size().reset_index(name='count')

,evol_req_min_happiness,count
0,160.0,58
1,220.0,25


#### Campo evol_req_need_overworld_rain

In [35]:
df_pkmn_evolution_transformar.groupby('evol_req_needs_overworld_rain').size().reset_index(name='count')

,evol_req_needs_overworld_rain,count
0,False,1569
1,True,5


In [36]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_needs_overworld_rain'] == True]

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
1694,704,goodra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,True,NaN,NaN,NaN,NaN,NaN,False,False
1697,705,goodra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,True,NaN,NaN,NaN,NaN,NaN,False,False
1700,706,goodra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,True,NaN,NaN,NaN,NaN,NaN,False,False
2844,10241,goodra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,True,NaN,NaN,NaN,NaN,NaN,False,False
2847,10242,goodra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,True,NaN,NaN,NaN,NaN,NaN,False,False


#### Campo evol_req_party_species

In [37]:
df_pkmn_evolution_transformar.groupby('evol_req_party_species').size().reset_index(name='count')

,evol_req_party_species,count
0,"{'name': 'remoraid', 'url': 'https://pokeapi.c...",2


In [38]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_party_species'] = df_pkmn_evolution_transformar['evol_req_party_species'].apply(extract_name_from_json)

In [39]:
df_pkmn_evolution_transformar.groupby('evol_req_party_species').size().reset_index(name='count')

,evol_req_party_species,count
0,remoraid,2


In [40]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_party_species'] == 'remoraid']

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
598,226,mantine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,remoraid,NaN,NaN,NaN,NaN,False,False
1129,458,mantine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,remoraid,NaN,NaN,NaN,NaN,False,False


#### Campo evol_req_party_type

In [41]:
df_pkmn_evolution_transformar.groupby('evol_req_party_type').size().reset_index(name='count')

,evol_req_party_type,count
0,"{'name': 'dark', 'url': 'https://pokeapi.co/ap...",2


In [42]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_party_type'] = df_pkmn_evolution_transformar['evol_req_party_type'].apply(extract_name_from_json)

In [43]:
df_pkmn_evolution_transformar.groupby('evol_req_party_type').size().reset_index(name='count')

,evol_req_party_type,count
0,dark,2


#### Campo evol_req_relative_physical_stats

In [44]:
df_pkmn_evolution_transformar.groupby('evol_req_relative_physical_stats').size().reset_index(name='count')

,evol_req_relative_physical_stats,count
0,-1.0,4
1,0.0,4
2,1.0,4


In [45]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_relative_physical_stats'] == -1.0]

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
279,106,hitmonchan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,-1.0,NaN,NaN,False,False
283,107,hitmonchan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,-1.0,NaN,NaN,False,False
619,236,hitmonchan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,-1.0,NaN,NaN,False,False
623,237,hitmonchan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,-1.0,NaN,NaN,False,False


In [46]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_relative_physical_stats'] == 1.0]

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
278,106,hitmonlee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,1.0,NaN,NaN,False,False
282,107,hitmonlee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,1.0,NaN,NaN,False,False
618,236,hitmonlee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,1.0,NaN,NaN,False,False
622,237,hitmonlee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,1.0,NaN,NaN,False,False


In [47]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_relative_physical_stats'] == 0.0]

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
280,106,hitmontop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,0.0,NaN,NaN,False,False
284,107,hitmontop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,0.0,NaN,NaN,False,False
620,236,hitmontop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,0.0,NaN,NaN,False,False
624,237,hitmontop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,False,NaN,NaN,0.0,NaN,NaN,False,False


#### Campo evol_req_time_of_day

In [48]:
df_pkmn_evolution_transformar.groupby('evol_req_time_of_day').size().reset_index(name='count')

,evol_req_time_of_day,count
0,day,37
1,full-moon,4
2,night,30


#### Campo evol_req_trade_species

In [49]:
df_pkmn_evolution_transformar.groupby('evol_req_trade_species').size().reset_index(name='count')

,evol_req_trade_species,count
0,"{'name': 'karrablast', 'url': 'https://pokeapi...",2
1,"{'name': 'shelmet', 'url': 'https://pokeapi.co...",2


In [50]:
df_pkmn_evolution_transformar.loc[:, 'evol_req_trade_species'] = df_pkmn_evolution_transformar['evol_req_trade_species'].apply(extract_name_from_json)

In [51]:
df_pkmn_evolution_transformar.groupby('evol_req_trade_species').size().reset_index(name='count')

,evol_req_trade_species,count
0,karrablast,2
1,shelmet,2


In [52]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_trade_species']=='karrablast']

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
1505,616,accelgor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,karrablast,False,False
1507,617,accelgor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,karrablast,False,False


In [53]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_trade_species']=='shelmet']

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
1439,588,escavalier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,shelmet,False,False
1441,589,escavalier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,shelmet,False,False


#### Campo evol_req_turn_upside_down

In [54]:
df_pkmn_evolution_transformar.groupby('evol_req_turn_upside_down').size().reset_index(name='count')

,evol_req_turn_upside_down,count
0,False,1572
1,True,2


In [55]:
df_pkmn_evolution_transformar[df_pkmn_evolution_transformar['evol_req_turn_upside_down']== True]

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
1653,686,malamar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,False,NaN,NaN,NaN,NaN,NaN,True,False
1655,687,malamar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,False,NaN,NaN,NaN,NaN,NaN,True,False


In [56]:
df_pkmn_evolution_transformar.head()

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
0,1,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False
2,1,venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False,False
3,2,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False


## Crear Tabla pkmn_primary_data_evolution_rel

In [57]:
df_temp = df_pkmn_primary_data_transformar[['pkmn_id', 'pokemon_name']]

In [58]:
df_pkmn_evolution_transformar.head()

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
0,1,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False
2,1,venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False,False
3,2,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False


In [59]:
df_pkmn_primary_data_evolution_rel_transformar = pd.merge(df_pkmn_evolution_transformar, 
                    df_temp, 
                    left_on='evolve_to_pkmn', 
                    right_on='pokemon_name', 
                    how='left')

In [60]:
df_pkmn_primary_data_evolution_rel_transformar.head()

,pkmn_id_x,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,...,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby,pkmn_id_y,pokemon_name
0,1,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.0,bulbasaur
1,1,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,False,2.0,ivysaur
2,1,venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,False,3.0,venusaur
3,2,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.0,bulbasaur
4,2,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,False,2.0,ivysaur


In [61]:
df_pkmn_primary_data_evolution_rel_transformar.rename(columns={'pkmn_id_x': 'pkmn_id', 
                            'pkmn_id_y': 'pkmn_id_evolve_to'}, inplace=True)

In [62]:
df_pkmn_primary_data_evolution_rel_transformar.head()

,pkmn_id,evolve_to_pkmn,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,...,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby,pkmn_id_evolve_to,pokemon_name
0,1,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.0,bulbasaur
1,1,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,False,2.0,ivysaur
2,1,venusaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,False,3.0,venusaur
3,2,bulbasaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,1.0,bulbasaur
4,2,ivysaur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,False,NaN,NaN,NaN,NaN,NaN,False,False,2.0,ivysaur


In [63]:
df_pkmn_primary_data_evolution_rel_transformar = df_pkmn_primary_data_evolution_rel_transformar[['pkmn_id', 'pkmn_id_evolve_to', 'evol_req_gender' , 'evol_req_held_item'
                       , 'evol_req_item', 'evol_req_known_move', 'evol_req_known_move_type'
                       , 'evol_req_location', 'evol_req_min_affection', 'evol_req_min_beauty'
                       , 'evol_req_min_happiness', 'evol_req_min_level'
                       , 'evol_req_needs_overworld_rain', 'evol_req_party_species'
                       , 'evol_req_party_type', 'evol_req_relative_physical_stats'
                       , 'evol_req_time_of_day', 'evol_req_trade_species'
                       , 'evol_req_turn_upside_down', 'evol_req_baby']]

In [64]:
df_pkmn_primary_data_evolution_rel_transformar[df_pkmn_primary_data_evolution_rel_transformar['pkmn_id'] == 133]

,pkmn_id,pkmn_id_evolve_to,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
339,133,133.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
340,133,134.0,NaN,NaN,water-stone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
341,133,135.0,NaN,NaN,thunder-stone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
342,133,136.0,NaN,NaN,fire-stone,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
343,133,196.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,NaN,False,NaN,NaN,NaN,day,NaN,False,False
344,133,197.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.0,NaN,False,NaN,NaN,NaN,night,NaN,False,False
345,133,470.0,NaN,NaN,NaN,NaN,NaN,eterna-forest,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
346,133,471.0,NaN,NaN,NaN,NaN,NaN,sinnoh-route-217,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False
347,133,700.0,NaN,NaN,NaN,NaN,fairy,NaN,2.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,False,False


In [66]:
df_pkmn_primary_data_evolution_rel_transformar.head(10)

,pkmn_id,pkmn_id_evolve_to,evol_req_gender,evol_req_held_item,evol_req_item,evol_req_known_move,evol_req_known_move_type,evol_req_location,evol_req_min_affection,evol_req_min_beauty,evol_req_min_happiness,evol_req_min_level,evol_req_needs_overworld_rain,evol_req_party_species,evol_req_party_type,evol_req_relative_physical_stats,evol_req_time_of_day,evol_req_trade_species,evol_req_turn_upside_down,evol_req_baby
0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False
2,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False,False
3,2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False
5,2,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False,False
6,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
7,3,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,False,NaN,NaN,NaN,NaN,NaN,False,False
8,3,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,False,NaN,NaN,NaN,NaN,NaN,False,False
9,4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


## Crear tabla relacion pokemon_primary_data_abilities_rel

In [67]:
# Generar una tabla de habilidades únicas
df_abilities_transformar = df_pkmn_abilities_transformar.copy()
df_abilities_transformar = df_abilities_transformar[['ability_id', 'ability_name', 'ability_description']].drop_duplicates().reset_index(drop=True)

In [68]:
df_abilities_transformar.head()

,ability_id,ability_name,ability_description
0,65,overgrow,Powers up Grass-type moves when the Pokémon's ...
1,34,chlorophyll,Boosts the Pokémon's Speed stat in harsh sunli...
2,66,blaze,Powers up Fire-type moves when the Pokémon's H...
3,94,solar-power,"In harsh sunlight, the Pokémon's Sp. Atk stat ..."
4,67,torrent,Powers up Water-type moves when the Pokémon's ...


In [69]:
df_pkmn_primary_data_abilities_rel = df_pkmn_abilities_transformar.copy()
df_pkmn_primary_data_abilities_rel = df_pkmn_primary_data_abilities_rel[['pkmn_id', 'ability_id', 'ability_is_hidden']]

In [70]:
df_pkmn_primary_data_abilities_rel.head()

,pkmn_id,ability_id,ability_is_hidden
0,1,65,False
1,1,34,True
2,2,65,False
3,2,34,True
4,3,65,False


## Crear tabla relación pkmn_primary_data_tipos_rel

In [71]:
df_pkmn_types_transformar.head()

,pkmn_id,name,type_number
0,1,grass,1
1,1,poison,2
2,2,grass,1
3,2,poison,2
4,3,grass,1


In [72]:
df_types_transformar = df_pkmn_types_transformar.copy()
df_types_transformar = df_types_transformar[['name']].drop_duplicates().reset_index(drop=True)

In [73]:
df_types_transformar.head()

,name
0,grass
1,poison
2,fire
3,flying
4,water


In [74]:
df_pkmn_primary_data_types_rel = df_pkmn_types_transformar[['pkmn_id', 'name','type_number']]

In [75]:
df_pkmn_primary_data_types_rel.head()

,pkmn_id,name,type_number
0,1,grass,1
1,1,poison,2
2,2,grass,1
3,2,poison,2
4,3,grass,1


## Crear Tabla relacion pkmn_primary_data_stats_rel

In [76]:
df_stats_transformar = df_pkmn_stats_transformar.copy()
df_stats_transformar = df_stats_transformar[['name']].drop_duplicates().reset_index(drop=True)

In [77]:
df_stats_transformar.head()

,name
0,hp
1,attack
2,defense
3,special-attack
4,special-defense


In [78]:
df_pkmn_primary_data_stats_rel = df_pkmn_stats_transformar[['pkmn_id', 'name', 'stat', 'effort']]

In [79]:
df_pkmn_primary_data_stats_rel.head()

,pkmn_id,name,stat,effort
0,1,hp,45,0
1,1,attack,49,0
2,1,defense,49,0
3,1,special-attack,65,1
4,1,special-defense,65,0


## Crear tabla relacion  pkmn_primary_data_pokedex_descripcion_version_juego_rel

In [80]:
df_pkmn_game_appears_transformar.head()

,pkmn_id,game_version
0,1,red
1,1,blue
2,1,yellow
3,1,gold
4,1,silver


In [81]:
df_game_transformar = df_pkmn_game_appears_transformar.copy()
df_game_transformar = df_game_transformar[['game_version']].drop_duplicates().reset_index(drop=True)

In [82]:
df_game_transformar.head()

,game_version
0,red
1,blue
2,yellow
3,gold
4,silver


In [83]:
df_pkmn_primary_data_game_appears_rel = df_pkmn_game_appears_transformar[['pkmn_id', 'game_version']]

In [84]:
df_pkmn_primary_data_game_appears_pokedex_description_rel = pd.merge(
    df_pkmn_primary_data_game_appears_rel, df_pkmn_pokedex_description_transformar[['pkmn_id', 'game_name', 'id_pokedex_description']]
    , left_on=['pkmn_id', 'game_version']
    , right_on=['pkmn_id', 'game_name']
    , how='inner'
)

In [85]:
df_pkmn_primary_data_game_appears_pokedex_description_rel = df_pkmn_primary_data_game_appears_pokedex_description_rel[['pkmn_id', 'game_version', 'id_pokedex_description']]

In [86]:
df_pkmn_primary_data_game_appears_pokedex_description_rel.head()

,pkmn_id,game_version,id_pokedex_description
0,1,red,1
1,1,blue,2
2,1,yellow,3
3,1,gold,4
4,1,silver,5


In [87]:
df_pokedex_description_transformar = df_pkmn_pokedex_description_transformar.copy()
df_pokedex_description_transformar = df_pokedex_description_transformar[['id_pokedex_description', 'flavor_text']]

In [88]:
df_pokedex_description_transformar['flavor_text'] = df_pokedex_description_transformar['flavor_text'].str.replace('\x0c', '', regex=False)

## Agregar Información de las imagenes Base64

In [89]:
# Agregar información de la imagen en base64 
# save_directory
data_imagenes = []
for image_name in os.listdir(save_directory):
    image_path = os.path.join(save_directory, image_name)

    with open(image_path, 'rb') as img_file:
        base64_string = base64.b64encode(img_file.read()).decode('utf-8')
        img_html = f'<img src="data:image/png;base64,{base64_string}" style="width:{300}px; height:{300}px;" alt="Image"/>'

    data_imagenes.append({
        'pkmn_id': image_name.split('-')[0],
        'image_name': image_name,
        'image_base64': base64_string,
        'img_html': img_html
    })

df_pkmn_images_transformar = pd.DataFrame(data_imagenes)

## Agregar tema de codigo de colores RBG

### Mapeo colores tabla df_Stats

In [90]:
hex_stats_mapping = {
    'hp': '#ff0000',
    'attack': '#f08030',
    'defense': '#f8d030',
    'special-attack': '#6890f0',
    'special-defense': '#78c850',
    'speed': '#f85888'
}

In [91]:
df_stats_transformar['color_hex'] = df_stats_transformar['name'].map(hex_stats_mapping)

In [92]:
df_stats_transformar.head()

,name,color_hex
0,hp,#ff0000
1,attack,#f08030
2,defense,#f8d030
3,special-attack,#6890f0
4,special-defense,#78c850


### Mapeo colores tabla df_types

In [93]:
hex_types_mapping = {
    'normal': '#A8A77A', 'fire': '#EE8130', 'water': '#6390F0', 'electric': '#F7D02C'
    , 'grass': '#7AC74C', 'ice': '#96D9D6', 'fighting': '#C22E28', 'poison': '#A33EA1'
    , 'ground': '#E2BF65','flying': '#A98FF3','psychic': '#F95587',	'bug': '#A6B91A'
    , 'rock': '#B6A136', 'ghost': '#735797', 'dragon': '#6F35FC', 'dark': '#705746'
    , 'steel': '#B7B7CE',	'fairy': '#D685AD'
}

In [94]:
df_types_transformar['color_hex'] = df_types_transformar['name'].map(hex_types_mapping)

### Mapeo colores tabla df_game_version

In [95]:
hex_game_version_mapping = {
'red': '#FF0000', 'blue': '#0000FF', 'yellow': '#FFFF00', 'gold': '#D2C21A'
, 'silver': '#A0A0A0', 'crystal': '#00B3F5', 'ruby': '#B80000', 'sapphire': '#0D00B8'
, 'emerald': '#11B800', 'firered': '#FF7200', 'leafgreen': '#4B9C0A', 'diamond': '#6077FF'
, 'pearl': '#FC72E4', 'platinum': '#6B6B6B', 'heartgold': '#D19A19', 'soulsilver': '#B8B8B8'
, 'black': '#424242', 'white': '#F2F2F2', 'black-2': '#424242', 'white-2': '#F2F2F2'
, 'omega-ruby': '#D4400F'
}

In [96]:
df_game_transformar['color_hex'] = df_game_transformar['game_version'].map(hex_game_version_mapping)

In [97]:
df_game_transformar.head()

,game_version,color_hex
0,red,#FF0000
1,blue,#0000FF
2,yellow,#FFFF00
3,gold,#D2C21A
4,silver,#A0A0A0


## Agregarle el DateTime a las tablas

In [98]:
df_pkmn_primary_data_transformar['transformar_datetime'] = datetime.now()
df_pkmn_primary_data_abilities_rel['transformar_datetime'] = datetime.now()
df_pkmn_primary_data_stats_rel['transformar_datetime'] = datetime.now()
df_pkmn_primary_data_types_rel['transformar_datetime'] = datetime.now()
df_pkmn_primary_data_game_appears_pokedex_description_rel['transformar_datetime'] = datetime.now()
df_pkmn_primary_data_evolution_rel_transformar['transformar_datetime'] = datetime.now()
df_abilities_transformar['transformar_datetime'] = datetime.now()
df_stats_transformar['transformar_datetime'] = datetime.now()
df_types_transformar['transformar_datetime'] = datetime.now()
df_game_transformar['transformar_datetime'] = datetime.now()
df_pkmn_images_transformar['transformar_datetime'] = datetime.now()


## Exportar Transformar 

In [99]:
df_pkmn_primary_data_transformar.to_csv('fuente/transformar/pkmn_primary_data.csv', sep = '|', index=False)
df_pkmn_primary_data_abilities_rel.to_csv('fuente/transformar/pkmn_primary_data_rel_abilities.csv', sep = '|', index=False)
df_pkmn_primary_data_stats_rel.to_csv('fuente/transformar/pkmn_primary_data_rel_stats.csv', sep = '|', index=False)
df_pkmn_primary_data_types_rel.to_csv('fuente/transformar/pkmn_primary_data_rel_types.csv', sep = '|', index=False)
df_pkmn_primary_data_game_appears_pokedex_description_rel.to_csv('fuente/transformar/pkmn_primary_data_rel_game_appears_pokedex_description.csv', sep = '|', index=False)
df_pkmn_primary_data_evolution_rel_transformar.to_csv('fuente/transformar/pkmn_primary_data_rel_evolution.csv', sep = '|', index=False)
df_abilities_transformar.to_csv('fuente/transformar/pkmn_abilities.csv', sep = '|', index=False)
df_stats_transformar.to_csv('fuente/transformar/pkmn_stats.csv', sep = '|', index=False)
df_types_transformar.to_csv('fuente/transformar/pkmn_types.csv', sep = '|', index=False)
df_game_transformar.to_csv('fuente/transformar/pkmn_game_appears.csv', sep = '|', index=False)
df_pkmn_images_transformar.to_csv('fuente/transformar/pkmn_images.csv', sep = '|', index=False)
df_pokedex_description_transformar.to_csv('fuente/transformar/pkmn_pokedex_description.csv', sep='|', index=False)

In [384]:
df_pokedex_description_transformar.head()

,id_pokedex_description,flavor_text
0,1,A strange seed was planted on its back at birt...
1,2,A strange seed was planted on its back at birt...
2,3,It can go for days without eating a single mor...
3,4,The seed on its back is filled with nutrients....
4,5,It carries a seed on its back right from birth...


# Cargar